# Создание агента с функциями на базе GigaChat

С помощью [нейросетевой модели GigaChat](https://developers.sber.ru/docs/ru/gigachat/overview) вы можете вызывать различные функции — инструменты, которые будет использовать модель для решения поставленной задачи.
Модель самостоятельно решает, когда и какой инструмент нужно использовать. 
Вызванные функции выполняются на стороне клиента.
Вы можете использовать функции как разработанные самостоятельно, так и доступные в GigaChain и сторонних библиотеках.

Для использования функций нужно создать агента с помощью модуля [`gigachat_functions_agent`](/libs/langchain/langchain/agents/gigachat_functions_agent/base.py).

Раздел содержит пример агента, который задает вопрос в поисковом сервисе DuckDuckGo и рисует ответ с помощью Ascii-графики. Итоговый агент не только умеет работать с разными инструментами, но и обладает памятью.

## Агент с одной функцией

Для создания агента с функцией обращения к поисковому сервису DuckDuckGo:

1. Инициализируйте GigaChat.
2. Добавьте функцию поиска в массив инструментов доступных модели.
3. Передайте агенту модель и массив инструментов.

### Инициализация GigaChat

Для работы агента создайте экземпляр класса GigaChat.
В классе укажите модель, поддерживающую работу с функциями:

In [1]:
from langchain.chat_models.gigachat import GigaChat

giga = GigaChat(
    credentials="авторизационные_данные", model="модель_с_поддержкой_функций"
)

Для авторизации запросов к GigaChat используйте данные, полученные при [создании проекта GigaChat API](https://developers.sber.ru/docs/ru/gigachat/api/integration).

Список и описание доступных моделей — в разделе [Модели GigaChat](https://developers.sber.ru/docs/ru/gigachat/models).

### Добавление функции поиска

Добавьте адаптер поискового сервиса [DuckDuckGo](https://www.duckduckgo.com) в список инструментов, которые будет использовать модель.

Для этого импортируйте класс `DuckDuckGoSearchRun` и добавьте функцию `DuckDuckGoSearchRun()` в массив `tools`:



In [2]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()
tools = [search_tool]

### Создание и запуск агента

Инициализируйте агента. Передайте ему модель, которая будет вызывать функции, и массив доступных ей инструментов `tools`:

In [3]:
from langchain.agents import AgentExecutor, create_gigachat_functions_agent

agent = create_gigachat_functions_agent(giga, tools)

# AgentExecutor создает среду, в которой будет работать агент
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
)

Запустите агента с помощью функции `invoke()`:

In [4]:
agent_executor.invoke(
    {"input": "Найди текущий курс биткоина и напечатай только число"}
)["output"]



> Entering new AgentExecutor chain...

Invoking: `duckduckgo_search` with `{'query': 'курс биткоина'}`


Курс Биткоина (BTC): цена Bitcoin, индекс, новости — Investing.com Новинка! 💥 Приобретите ProPicks и узнайте стратегию, опередившую S&P 500 более чем на 1,183% Получить скидку 40% Биткоин... 48 306,99 $ 1.2% 1,0000 BTC 0.0% 47 623,87 $ Диапазон за 24 ч 48 487,28 $ Добавить в портфель • 1 453 248 added Конвертируйте Биткоин в US Dollar (BTC в USD) Сегодня цена конвертации 1 Биткоин (BTC) в USD равна $48 324,27. BTC USD 1 BTC = 48 306,99 $ Как купить BTC за USD Показать График BTC в USD Сегодня Биткоин (BTC) стоит $48 324,27. Price of BTC today The live price of Bitcoin is $ 45,464.14 per (BTC / USD) with a current market cap of $ 892.11B USD. 24-hour trading volume is $ 26.67B USD. BTC to USD price is updated in real-time. Bitcoin is +2.55% in the last 24 hours with a circulating supply of 19.62M. BTC Price History USD BTC Price Information 24h Low & High Курс криптовалют онлайн - 

'48 324,27'

## Агент с несколькими функциями

Для создания агента, который кроме поиска будет возвращать текст в виде Ascii-графики, вам потребуется библиотека pyfiglet.
Вы можете установить ее с помощью команды:

```sh
pip install pyfiglet
```

Библиотека умеет работать только с латинскими символами и цифрами.

Пример работы pyfiglet:

In [5]:
import pyfiglet

pyfiglet.print_figlet("Hello!", font="epic")

          _______  _        _        _______  _ 
|\     /|(  ____ \( \      ( \      (  ___  )( )
| )   ( || (    \/| (      | (      | (   ) || |
| (___) || (__    | |      | |      | |   | || |
|  ___  ||  __)   | |      | |      | |   | || |
| (   ) || (      | |      | |      | |   | |(_)
| )   ( || (____/\| (____/\| (____/\| (___) | _ 
|/     \|(_______/(_______/(_______/(_______)(_)
                                                



### Использование нескольких функций

Чтобы упростить создание функций из python-кода, используйте декоратор `@tool`.
Он преобразует любую функцию в инструмент, доступный модели для вызова.

:::note

Модель ориентируется как на название и описание функции, так и на описание и типы аргументов возвращаемого значения.
Чтобы модель правильно понимала, как нужно использовать инструмент, все значения функции нужно явно указать.

:::

Создайте функцию `draw_banner()`, которая будет возвращать результат поиска в виде Ascii-графики, и добавьте ее в новый массив инструментов `new_tools`:

In [13]:
from langchain.tools import tool


@tool
def draw_banner(number: str) -> str:
    """Рисует баннер с текстом результатов кода в виде Ascii-графики

    Args:
        number (str): Число, которое нужно нарисовать на баннере
    """
    pyfiglet.print_figlet(number, font="epic")
    return "Draw complete"


new_tools = [search_tool, draw_banner]

Инициализируйте и запустите агента с просьбой найти нужное значение и нарисовать его на баннере:

In [15]:
agent = create_gigachat_functions_agent(giga, new_tools)

agent_executor = AgentExecutor(
    agent=agent,
    tools=new_tools,
    verbose=True,
)

agent_executor.invoke(
    {
        "input": "Найди в интернете курс биткоина в долларах и нарисуй это число на банере."
    }
)["output"]



> Entering new AgentExecutor chain...

Invoking: `duckduckgo_search` with `{'query': 'курс биткоина в долларах'}`


Сегодня цена конвертации 1 Биткоин (BTC) в USD равна $48 324,27. BTC USD 1 BTC = 48 306,99 $ Как купить BTC за USD Показать График BTC в USD Сегодня Биткоин (BTC) стоит $48 324,27. Это на 0.5% увеличилась, чем час назад, и на 1.2% увеличилась, чем вчера. Стоимость BTC сегодня на 13.5% больше, чем 7 дней назад. 1 BTC = 47 384,89 USD по курсу на 10.02.2024. Стоимость 1 биткоина в долларах США на сегодня составляет 47 384,89 $ по данным ЦБ РФ, по сравнению со вчерашним днём курс валюты увеличился на 0,16% (на +73,73 $). Курс биткоина по отношению к доллару США на графике, таблица динамики стоимости в процентах за день, неделю, месяц и год. Конвертер валют Узнайте курс биткоина в долларах в разных банках и обменниках, а также динамику их изменений за месяц и год. Сравните курсы биткоина в долларах с другими валютами и посмотрите средние курсы в банках, Межбанке, НБУ, Visa и

'- вот баннер с курсом биткоина в долларах.'

## Добавление памяти

В созданном агенте отсутствуют данные о состоянии, таким образом, он не помнит предыдущие взаимодействия с пользователем.

Чтобы агент помнил, о чем пользователь разговаривал с ним, передайте историю сообщений в переменной `chat_history`:

In [20]:
from langchain_core.messages import AIMessage, HumanMessage

agent_executor = AgentExecutor(
    agent=agent,
    tools=new_tools,
    verbose=False,
)

agent_executor.invoke(
    {
        "chat_history": [
            HumanMessage(
                content="Привет! Запомни трех животных - слон, жираф, крокодил"
            ),
            AIMessage(content="Привет! Хорошо, я запомнил."),
        ],
        "input": "Что я просил тебя запомнить?",
    }
)

{'chat_history': [HumanMessage(content='Привет! Запомни трех животных - слон, жираф, крокодил'),
  AIMessage(content='Привет! Хорошо, я запомнил.')],
 'input': 'Что я просил тебя запомнить?',
 'output': 'Вы просили меня запомнить трех животных: слон, жираф и крокодил.'}

### Использование памяти в чате с агентом

Вы можете использовать память для сохранения истории и промежуточных результатов общения пользователя с агентом:

In [21]:
chat_history = []
while True:
    user_input = input("Вы: ")
    print(f"Пользователь: {user_input}")
    if user_input == "":
        break
    result = agent_executor.invoke(
        {
            "chat_history": chat_history,
            "input": user_input,
        }
    )
    chat_history.append(HumanMessage(content=user_input))
    chat_history.append(AIMessage(content=result["output"]))
    print(f"Агент: {result['output']}")

User: Привет
Bot: Здравствуйте! Могу ли я помочь Вам с чем-то?
User: Найди возраст Илона Маска
Bot: Илон Маск родился 28 июня 1971 года, поэтому на текущий момент (2022 год) ему 50 лет.
User: Выведи его возраст на банер
 _______  _______ 
(  ____ \(  __   )
| (    \/| (  )  |
| (____  | | /   |
(_____ \ | (/ /) |
      ) )|   / | |
/\____) )|  (__) |
\______/ (_______)
                  

Bot: Возраст Илона Маска - 50 лет.
User: 
